In [12]:
import copy

In [13]:
#----------------------------   Constantes -------------------------------------

BLANC = chr(9788)
NOIR = chr(9787)
VIDE = "."
COULEURS = { BLANC : chr(9786) , NOIR : chr(9787), VIDE : "." }

#----------------------------   Classe Case -------------------------------------

class Case(object):
    def __init__(self, position, couleur=VIDE):
        if couleur not in (VIDE, NOIR, BLANC):
            pass
        else:
            self.__position = position
            self.__valeur = str(couleur)
            self.__voisinage = {}
            self.__voisins = []

            self.__estBloquee = False
            if self.__valeur != VIDE:
                self.__estLibre = False
                if len(self.__voisins)==0:
                    self.__estBloquee = True
            else:
                self.__estLibre = True


    # Getter && Setter

    @property
    def position(self):
        return self.__position

    @property
    def valeur(self):
        return self.__valeur

    @valeur.setter
    def valeur(self, x):
        if(x !=BLANC and x !=NOIR and x !=VIDE):
            pass
        else:
            if x != VIDE:
                self.__valeur = x
                self.__estBloquee = True
                self.__estLibre = False
                for y in self.__voisins:
                    if y.valeur == VIDE:
                        self.__estBloquee = False
                    y.__estBloquee = True
                    for z in y.__voisins:
                        if z.__estLibre:
                            y.__estBloquee = False
            else:
                self.__valeur = VIDE
                self.__estLibre = True
                self.__estBloquee = False


    @property
    def voisinage(self):
        return self.__voisinage.copy()

    @property
    def voisins(self):
        return self.__voisins

    @property
    def estLibre(self):
        return self.__estLibre

    @property
    def estBloquee(self):
        if self.valeur != VIDE:
            self.__estBloquee = True
            for v in self.voisins:
                if v.__estLibre == True:
                    self.__estBloquee = False
        return self.__estBloquee

    @voisins.setter
    def voisins(self, tupleIndexCase):
        if (len(tupleIndexCase)==2):
            index, case = tupleIndexCase
            if(isinstance(case, Case)):
                if(index in [0,1,2,3,4,5,6,7]):
                    if (tupleIndexCase) not in self.__voisinage:
                        self.__voisins.append(case)
                        self.__voisinage[index] = case
                        calcul = ((index+4)%8)
                        case.voisins.append(self)
                        case.__voisinage[calcul] = self




    # Méthodes

    def __str__(self):
        return COULEURS[self.__valeur]
    
    def __repr__(self):
        return "Case : position({}), couleur({})".format(self.__position, self.__valeur)

    def estAccessible(self, couleur):
        isTrue = False
        if couleur == VIDE:
            pass
        else:
            if self.__valeur == VIDE:
                for c in self.__voisins:
                    if c.__valeur == couleur:
                        isTrue = True

        return isTrue

    def force(self, couleur):
        cpt = 0
        if couleur == VIDE:
            pass
        else:
            if self.estAccessible(couleur):
                for v in self.voisins:
                    if v.valeur != VIDE:
                        if v.valeur != couleur:
                            cpt += 1
        return cpt





#----------------------------   Classe Plateau -------------------------------------

class Plateau(object) :
    def __init__(self, x=5, h=5):
        self.__largeur=x
        self.__hauteur=h
        self.__dictionnaireDesCases = {}
        self.__configuration = []

        if self.__hauteur%2==0:
            self.__hauteur=5
            

        # Initialisation des cases
        fin = (self.__hauteur*self.__largeur)+((self.__hauteur)//2)**2
        for i in range (fin):
            x = Case(i,VIDE)
            self.__configuration.append(x)


        m = self.__hauteur // 2


        # Initialisation du dictionnaire, dico = position : coordonées
        dico = []
        largeur = self.__largeur

        for i in range(self.__hauteur):
            for j in range(largeur):
                tupleCoordonee = (i,j)
                dico.append(tupleCoordonee)

            if i < m:
                largeur += 1

            elif largeur > self.__largeur:
                largeur -= 1

        for case in range(len(self)):
            self.__dictionnaireDesCases[case] = dico[case]


        # Initialisation des voisins

        largeur = self.__largeur
        for i in range(self.hauteur):
            compt = 0
            for j in range(largeur - 1):
                self[i, j].voisins = 2, self[i, j + 1]
            if i < m:
                compt += 1
                for x in range(largeur):
                    self[i, x].voisins = 5, self[i + 1, x]
                    self[i, x].voisins = 3, self[i + 1, x + 1]
                largeur += 1

            if i > m:
                compt += 1
                largeur -= 1
                for y in range(largeur + 1):
                    self[i, y].voisins = 7, self[i - 1, y]
                    self[i, y].voisins = 1, self[i - 1, y + 1]

            if compt == 0:
                largeur -= 1


        # Initialisations des cases libres 

        self.__libres = []
        for c in self.__configuration:
            if c.valeur == VIDE:
                self.__libres.append(c)



    #Getter && Setter

    @property
    def largeur(self):
        return self.__largeur

    @property
    def hauteur(self):
        return self.__hauteur

    @property
    def configuration(self):
        
        return copy.deepcopy(self.__configuration)
    

    @property
    def libres(self):
        caseLibre = []
        for c in self.__configuration:
            if c.valeur == VIDE:
                caseLibre.append(c)
        return caseLibre

    @property
    def dictionnaireDesCases(self):
        return self.__dictionnaireDesCases.copy()

    # Méthodes 

    def __str__(self):
        x = "Le plateau est de hauteur : " + str(self.hauteur) + " et de largeur : " + str(self.__largeur) + ". Il contient un total de : " + str((self.hauteur*self.__largeur)+((self.__hauteur)//2)**2) + " cases."
        return x

    def __len__(self):
        return (self.hauteur * self.__largeur) + ((self.__hauteur) // 2) ** 2

    def __repr__(self):
        return "Plateau : largeur({}), hauteur({}), len({}) ".format(self.__largeur, self.__hauteur, len(self))

    def __getitem__(self, key):
        obj = ""
        if (isinstance(key, tuple)):
            x, y = key
            obj = self.__configuration[self.coord2pos(x,y)]
        elif (isinstance(key, int)):
            obj = self.__configuration[key]

        return obj


    def pos2coord(self,pos):
        if pos not in range (len(self)):
            return None
            
        x = 0
        y = 0
        cpt = 0
        m = (self.__hauteur//2) + 1
        var = self.__largeur - 1
        for i in range(len(self.__configuration)):
            if i == pos:
                break
            y+= 1
            if i >= var:
                x+=1
                y=0
                if x<m:
                    cpt += 1
                    var = i + cpt + self.__largeur
                    
                if x>m or x==m:
                    cpt -= 1
                    var = i + cpt + self.__largeur
        coord = x,y
        return coord


    def coord2pos(self,x,y):
        position = [c for c,v in self.__dictionnaireDesCases.items() if v==(x,y)]
        return position[0]
                    
    def jouables(self,couleur):
        coupsjouables=[]
        for i in self.__configuration:
            if i.estAccessible(couleur) == True and i.estLibre == True:
                if i not in coupsjouables:
                    coupsjouables.append(i)

        return coupsjouables


    def jouablesTriees(self,couleur):
        return sorted(self.jouables(couleur), key=lambda case: case.force(couleur), reverse=True)


    def evaluation(self, couleur):
        if couleur == NOIR:
         adversaire = BLANC 
        else :
         adversaire = NOIR
         
        cpt = 0
        for c in self.jouables(couleur):
            cpt = cpt + 1 + (c.force(couleur)*4)
        for c in self.configuration :
            if c.valeur == couleur:
                cpt += 1
            elif c.valeur == adversaire:
                cpt -= 1
        return cpt
        



    def load(self, liste):
        cpt = 0
        if len(liste) == len(self):
            for i in liste:
                if i in COULEURS:
                    cpt += 1
            if cpt == len(liste):
                for i, x in enumerate(liste):
                    self[i].valeur = x








In [14]:

#----------------------------   Classe HexVirus -------------------------------------

class HexVirus():
   def __init__(self, largeur=5, hauteur=5):
        if hauteur % 2 == 0:
           hauteur = 5
        p = Plateau(largeur, hauteur)
        self.__plat = p
        self.reset()
        
    #Getter and Setter

   @property
   def plat(self):
       return self.__plat

   @property
   def trait(self):
       return self.__trait

   @trait.setter
   def trait(self,joueur):
      self.__trait = joueur

   @property
   def configuration(self):
       return ([case.valeur for case in self.__plat], self.__trait)

   @configuration.setter
   def configuration(self, tuplePlateauTrait):
       if type(tuplePlateauTrait) == tuple:
         plat, trait = tuplePlateauTrait
         if len(plat) == len(self.__plat) and trait in(COULEURS):
          self.__plat.load(plat)
          self.__trait = trait

    # Méthodes 

   def reset(self):
        self.__plat.load([VIDE]*len(self.__plat))
        self.__plat[0].valeur = BLANC
        self.__plat[self.__plat.hauteur-1, self.__plat.largeur-1].valeur = BLANC
        self.__plat[0, self.__plat.largeur-1].valeur = NOIR
        self.__plat[self.__plat.hauteur-1, 0].valeur = NOIR
        self.__trait = BLANC
        
   def __str__(self):
       graphique = ""
       largeur = self.__plat.largeur

       m = self.__plat.hauteur // 2
       for x in range(self.__plat.hauteur):
           distance = abs(x-m)

           graphique += "\n"
           graphique += distance * '  '
           for y in range(largeur):
               graphique += 3 * " "
               if self.__plat[x,y].valeur == VIDE:
                  graphique += str(self.__plat[x,y].position)
               else:
                  graphique += self.__plat[x,y].valeur

           if x < m:
             largeur += 1

           elif x > m:
             largeur -= 1

           elif x == m:
             largeur -= 1

       graphique += 2*"\n"
       graphique += "Le joueur " + str(self.__trait) + " a le trait"
       return graphique

   def regles(cls):
        """ affiche les regles du jeux """
        _msg = "REGLES : \n" \
               "1) Chaque joueur, à tour de role, peut poser un pion sur une des cases vides adjacentes à un de ses pions." \
               "\n" \
               "2) Lorsqu'un pion d'une couleur est posé à un endroit, tous les pions voisins de la couleur ennemi sont capturés " \
               "\n" \
               "(transformés dans la couleur du pion posé)." \
               "\n" \
               "3) La partie s'arrete lorsque toutes les cases vides sont remplies ou lorsque les joueurs n'ont plus de coups jouables." \
               "" \
               ""
        return _msg
      
   def adversaire(self, joueur=BLANC):
      """ renvoie l'autre joueur """
      if joueur not in (BLANC, NOIR): return None
      if joueur == NOIR:
         adversaire = BLANC
         
      else :
         adversaire = NOIR

      return adversaire

   def gagnant(self, joueur):
       if self.finPartie(joueur) or self.finPartie(self.adversaire(joueur)):
           j1 = 0
           j2 = 0
           for c in self.configuration[0]:
               if c == joueur: j1 +=1
               if c == self.adversaire(joueur): j2 += 1
           if j1 > j2: return True
       return False

   def perdant(self, joueur):
       return self.gagnant(self.adversaire(joueur))

   def finPartie(self, joueur):
      """ renvoie True si la partie est terminee """
      if len(self.__plat.libres) == 0 or len(self.__plat.jouables(joueur))== 0 or len(self.__plat.jouables(self.adversaire(joueur)))== 0:
         return True
      else :
         return False

   def listeCoups(self, joueur):
        """ renvoie la liste des coups autorises pour le joueur """
        return [case.position for case in self.__plat.jouables(joueur)]

   def joue(self, coup, joueur):
      #if joueur != self.__trait: return self.configuration
      newboard = self.configuration[0]
      trait = joueur
      if type(coup)== int :
         if type(coup) == Case : coup = Case[0]

         if coup not in self.listeCoups(joueur) or self.__trait != joueur:
            return self.configuration
         else:
            if coup in self.listeCoups(joueur):
               newboard[coup] = joueur
               for vois in self.__plat[coup].voisins :
                  if vois.valeur != VIDE :
                     newboard[vois.position] = joueur
      return newboard, self.adversaire(joueur)

   def evaluation(self, couleur):
        return self.__plat.evaluation(couleur)
    
#----------------------------   Classe Humain -------------------------------------
class Humain():
   def __init__(self): self.nom = None
   
   def __str__(self): return self.nom

   def __call__(self,*args,**kwargs):
        return self.choixCoup(*args,**kwargs)

   def GetName(self): self.nom = input('Entrez votre nom: ')

   def choixCoup(self, jeu, joueur):
      if joueur in {BLANC, NOIR}:
         if len(jeu.plat.jouables(joueur)) != 0:
            listeCoup = []
            print(jeu)
            print("Choisissez une case dans la liste suivante : ")
            string = ""
            for c in jeu.plat.jouables(joueur):
                string += " " + str(c.position) + " "
                listeCoup.append(c.position)
            print(string)
            toReturn = int(input("Quel coup choisissez-vous? :  "))
            if toReturn not in listeCoup:
                toReturn = None
         return toReturn
      else:
          return None




# ===========================================
def partie(jeu=HexVirus(), playerA= Humain(), playerB=Humain(), nbManchesIn=2):
    nbManches = 2
    if nbManchesIn % 2 == 0:
        nbManches = nbManchesIn
    print(jeu.regles(), "\n")
    x = 0
    scoreJ1 = 0
    scoreJ2 = 0
    while (x != nbManches):
        score = manche(jeu, playerA, playerB)
        J1 = score[0]
        J2 = score[1]
        if J1 > J2:
            scoreJ1 += 1
        elif J2 > J1:
            scoreJ2 += 1
        else:
            scoreJ1 += 0.5
            scoreJ2 += 0.5
        if x%2 == 0 :
            score = scoreJ1
            scoreJ1 = scoreJ2
            scoreJ2 = score
        x += 1
    print("Player A:", scoreJ1, "Player B:", scoreJ2)
    return (scoreJ1, scoreJ2)

def manche(jeu, playerA=Humain(), playerB=Humain()):
    playerA.couleur = BLANC
    playerB.couleur = NOIR
    print("Nouvelle manche : \n")
    joueur = None
    scoreJ1 = 0
    scoreJ2 = 0
    print("joueur 1:", playerA.couleur, "joueur 2: ", playerB.couleur)

    while jeu.finPartie(jeu.trait) == False:
        if jeu.trait == playerA.couleur:
            joueur = playerA
        else:
            joueur = playerB
        jeu.configuration = jeu.joue(joueur(jeu,jeu.trait), jeu.trait)
    for c in jeu.plat:
        if jeu.finPartie(playerA.couleur):
            if c.valeur == VIDE :
                scoreJ1 += 1
        elif jeu.finPartie(playerB.couleur):
            if c.valeur == VIDE :
                scoreJ2 += 1
        if c.valeur != VIDE:
            if c.valeur == playerA.couleur:
                scoreJ1 += 1
            else:
                scoreJ2 +=1
    
    score = (scoreJ1, scoreJ2)
    print("Score de la manche :", score)
    jeu.reset()
    return score

class Aleatoire():
   def __init__(self): self.nom = "Random"

   def choixCoup(self, jeu, joueur) : 
      if joueur in {BLANC, NOIR}:
         if len(jeu.plat.jouables(joueur)) != 0:
            coupChoisi = (choice(jeu.plat.jouables(joueur))).position
            return coupChoisi
      else: 
          return None

class Premier() :
   def __init__(self,who:str=''):
      self.nom = who
   def choixCoup(self, jeu, joueur):
      if joueur not in (BLANC, NOIR):return None
      _=jeu.jouablesTriees(joueur)
      if _ == [] : return None
      return _[0]


In [15]:

class IAPlayer():

    def __init__(self, lvl = 0, code = 0, mute = False):
        self.__name = 'IA'
        self.__lvl = lvl
        if lvl > 4:
            self.__lvl = 0
        self.__code = code
        self.__mute = mute

    def __str__(self): return self.nom

    def __call__(self,*args,**kwargs):
        return self.choixCoup(*args,**kwargs)

    
    @property
    def lvl(self):
       return self.__lvl

    @lvl.setter
    def lvl(self,n):
        if n > 4:
            self.__lvl = 0
        else :
            self.__lvl = n

    @property
    def code(self):
       dic = { 0 : "minmax", 1 : "minmaxIter", 2 : "negamax", 3 : "minmax", 4 : "alphabeta", 5 : "alphabetaNegamax", 6 : "minmax" }
       return dic[self.__code]

    @code.setter
    def code(self,n):
        self.__code = n


    @property
    def mute(self):
        return self.__mute
            
    @property
    def name(self):
       return self.__name

    def choixCoup(self, jeu, joueur):
        dicLvl = { 0 : 1, 1 : 2, 2 : 4, 3 : 6, 4 : 9 }
        x = Parcours(jeu)
        lvl = dicLvl[self.__lvl]
        code = self.code
        verbose = False
        if self.__mute == True :
            verbose = False
        else :
            verbose = True
        if verbose :
            print('je choisi  ', x.run(joueur, lvl, "minmax", verbose)[0], 'que jestime ' ,x.run(joueur, lvl, "minmax", verbose)[1])
        return x.run(joueur, lvl, "minmax", verbose)[0]

In [16]:
class Parcours(object):
    """ la classe qui permet de parcourir un arbre """
    BIGVALUE = 1e5 # l'infini
    def __init__(self, unJeu):
        self.__game = None
        self.__player = None
        self.__checkType(unJeu)

    def __checkType(self, unJeu):
        """
           On vérifie que l'on dispose des bons attributs
        """
        if isinstance(unJeu, HexVirus): self.__game = unJeu
        else:
            _lattrs = """
 listeCoups joue adversaire configuration
 etat_jeu trait finPartie evaluation
""".split()
            for attr in _lattrs:
                try:
                    assert hasattr(unJeu, attr), "{} is required"
                except Exception as _e:
                    raise ValueError(_e)
            self.__game = unJeu

    def run(self, joueur, pf, meth="minmax", verbose=False):
        """ 
        - joueur: le joueur qui pose la question
        - pfmax: la profondeur maximale à explorer
        - code: le code de la méthode à utiliser
        - verbose: quand vous voulez faire des print
        """
        methodes = {'minmax': (0,),
                    'minmaxIter': (0,),
                    'negamax': (0,),
                    'negamaxIter': (0,),
                    'alphabeta': (0,-self.BIGVALUE, self.BIGVALUE),
                    'alphabetaNegamax': (0,-self.BIGVALUE, self.BIGVALUE),
                    'alphabetaIter': (0,-self.BIGVALUE, self.BIGVALUE),
                    'alphabetaMemory': (0,-self.BIGVALUE, self.BIGVALUE),
                    'positionGagnante': (0,),
                    'positionPerdante': (0,),
                    'sss': (0,),
                    }
        assert  meth in methodes, "{} unknown"
        self.__player = joueur
        return getattr(self, meth)(*methodes[meth],
                                   jtrait=joueur, pfmax=pf, verbose=verbose)

    @property
    def moi(self): return self.__player
    @property
    def jeu(self): return self.__game
    def __getattr__(self,attr):
        """ un attribut inconnu est un attribut du jeu """
        return getattr(self.__game, attr)

    
    #### les méthodes ####
    def minmax(self, pf, jtrait=None, pfmax=1, verbose=False):
        """
        @return un coup, une évaluation

        Votre résultat est identique pour les 7 méthodes 
        """
        
        
        if self.moi is None: self.__player = jtrait # en cas d'appel direct

        #Je commence ici ----------

        if self.finPartie(jtrait) or pf == pfmax:
            if self.moi == jtrait:
                return None, self.evaluation(jtrait)
            else:
                return None, - self.evaluation(jtrait)
        else:
            _etat_entrant = self.configuration
            _adv = self.adversaire(jtrait)
            lfis = self.listeCoups(jtrait)
            if self.moi == jtrait:
                best = None, - self.BIGVALUE
                for f in lfis:
                    self.jeu.configuration = self.joue(f, jtrait)
                    _, ev = self.minmax(pf+1, _adv, pfmax, verbose)
                    self.jeu.configuration = _etat_entrant
                    if best[0] == None or ev > best[1]:
                        best = f, ev
            else:
                best = None, + self.BIGVALUE
                for fi in lfis:
                    self.jeu.configuration = self.joue(fi, jtrait)
                    _, ev = self.minmax(pf+1, _adv, pfmax, verbose)
                    self.jeu.configuration = _etat_entrant
                    if best[0] == None or ev < best[1]:
                        best = fi, ev
            return best   

    def minmaxIter(self, pf, jtrait=None, pfmax=1, verbose=False):
        """ minmax Iteratif """
        if self.moi is None: self.__player = jtrait # en cas d'appel direct
            
        _todo = Stack()
        _etat_entrant = self.etat_jeu, self.trait
        if self.finPartie(jtrait) or pf == pfmax:
            _ev = self.evaluation(jtrait)
            if jtrait != self.moi: _ev = -_ev
            _nfils = []
            _nval = _ev
        else:
            _nfils = self.listeCoups(jtrait)
            if jtrait == self.moi: _nval = -self.BIGVALUE
            else: _nval = self.BIGVALUE

        # on empile
        _todo.push(StackNode(None, *_etat_entrant, pf=pf,
                             fils=_nfils, bestFils=None, bestVal=_nval))

        # Boucle de calcul
        i = 0
        while not _todo.isEmpty:
            if verbose: print(i, _todo)
            _node = _todo.pop() # sommet à traiter
            if _node.fils == []: # fin du niveau
                # retour à l'appelant
                if _todo.isEmpty: #fin
                    self.jeu.configuration = _etat_entrant
                    print("*** nb push: {} ***".format(_todo.nbPush))
                    return _node.best
                else:
                    _pere = _todo.pop()
                    # mise à jour en fonction du MIN MAX
                    if _pere.player == self.moi: # MAX
                        if (_pere.best[0] is None or
                            _node.value > _pere.value):
                            _pere.best = _node.sommet, _node.value

                    else: #MIN
                        if (_pere.best[0] is None or
                            _node.value < _pere.value):
                            _pere.best = _node.sommet, _node.value

                    # restockage du pere
                    _todo.push(_pere)

            else: # autre fils à visiter
                _next = _node.fils.pop(0) # prochain sommet
                # construction des infos utles
                self.jeu.configuration = _node.cfg
                _nstate = self.joue(_next, _node.player)
                _npf = _node.pf + 1

                # est-on sur une feuille
                self.jeu.configuration = _nstate
                if self.finPartie(_nstate[1]) or _npf == pfmax:
                    _ev = self.evaluation(_nstate[1])
                    if _nstate[1] != self.moi: _ev = -_ev
                    _nfils = []
                    _nval = _ev
                else:
                    _nfils = self.listeCoups(_nstate[1])
                    if _nstate[1] == self.moi: _nval = self.BIGVALUE 
                    else: _nval = - self.BIGVALUE

                # On rempile l'appelant
                _todo.push(_node)
                # On empile l'appelé
                _todo.push(StackNode(_next, *_nstate, pf=_npf,
                                     fils=_nfils, bestFils=None, bestVal=_nval))
            i += 1
            if verbose: input() # pour voir l'évolution pas à pas
        

    def negamax(self, pf, jtrait=None, pfmax=1, verbose=False):
        """
        @return un coup, une évaluation
        Votre résultat est identique pour les 7 méthodes 
        """
        if self.moi is None: self.__player = jtrait # en cas d'appel direct

        if self.finPartie(jtrait) or pf == pfmax:
                return None, self.evaluation(jtrait)
        else:
            _etat_entrant = self.configuration
            _adv = self.adversaire(jtrait)
            lfis = self.listeCoups(jtrait)
            best = None, - self.BIGVALUE
            for f in lfis:
                self.jeu.configuration = self.joue(f, jtrait)
                _, ev = self.negamax(pf+1, _adv, pfmax, verbose)
                ev = - ev
                self.jeu.configuration = _etat_entrant
                if best[0] == None or ev > best[1]:
                    best = f, ev
            return best


        #raise NotImplemented
    

    def alphabeta(self, pf, alpha=-1000, beta=1000, jtrait=None, pfmax=1, verbose=False):
        """
        @return un coup, une évaluation
        Votre résultat est identique pour les 7 méthodes 
        """
        #-------COMMENCE ICI----------
        
        if self.finPartie(jtrait) or pf == pfmax:
            if self.moi == jtrait:
                return None, self.evaluation(jtrait)
            else:
                return None, - self.evaluation(jtrait)
        else:
            _etat_entrant = self.configuration
            _adv = self.adversaire(jtrait)
            lfis = self.listeCoups(jtrait)
            if self.moi == jtrait:
                best = None, - self.BIGVALUE
                i = 1
                while i <= len(lfis) and alpha < beta :
                    self.jeu.configuration = self.joue(lfis[i-1], jtrait)
                    coup, eval = self.alphabeta(pf + 1, alpha, beta, _adv, pfmax, verbose=False)
                    self.jeu.configuration = _etat_entrant
                    if eval > alpha:
                        alpha = eval
                        best = lfis[i-1], eval
                    i += 1
                return best
                    
            else:
                best = None, + self.BIGVALUE
                i = 1
                while i <= len(lfis) and alpha < beta :
                    self.jeu.configuration = self.joue(lfis[i-1], jtrait)
                    coup, eval = self.alphabeta(pf + 1, alpha, beta, _adv, pfmax, verbose=False)
                    self.jeu.configuration = _etat_entrant

                    if eval < beta:
                        beta = eval
                        best = lfis[i-1], eval
                    i += 1
                return best
        

    def alphabetaNegamax(self, pf, alpha=-1000, beta=1000, jtrait=None, pfmax=1,
                         verbose=False):
        if self.moi is None: self.__player = jtrait # en cas d'appel direct
        if self.finPartie(jtrait) or pf == pfmax:
                return None, self.evaluation(jtrait)
        else:
            _etat_entrant = self.configuration
            _adv = self.adversaire(jtrait)
            lfis = self.listeCoups(jtrait)
            best = None, - self.BIGVALUE
            i = 1
            while i <= len(lfis) and alpha < beta:
                self.jeu.configuration = self.joue(lfis[i-1], jtrait)
                coup, eval = self.alphabetaNegamax(pf + 1, -beta, -alpha, _adv, pfmax, verbose=False)
                eval = -eval
                self.jeu.configuration = _etat_entrant
                if eval > best[1]:
                    best = lfis[i-1], eval
                    if best[1] > alpha:
                        alpha = best[1]
                        if alpha >= beta:
                            return best
                i += 1
            return best


        #raise NotImplemented
    

    def positionGagnante(self, pf, jtrait=None, pfmax=1, verbose=False):
        """ @return coup, bool """
        if self.moi is None: self.__player = jtrait # en cas d'appel direct

        best = self.minmax(pf, jtrait, pfmax, verbose)
        if self.finPartie(jtrait) or pf == pfmax :
            if self.gagnant(jtrait) : 
                return None,True
            
        elif pf != pfmax :
            _adv = self.adversaire(jtrait)
            if self.positionPerdante(pf+1, _adv, pfmax, verbose)[1] :
                return best[0], True
        best = self.minmax(pf, jtrait, pfmax, verbose)   
        return best[0], False
        
        



    def positionPerdante(self, pf, jtrait=None, pfmax=1, verbose=False):
        """ @return coup, bool """
        if self.moi is None: self.__player = jtrait # en cas d'appel direct

        best = self.minmax(pf, jtrait=None, pfmax=1, verbose=False)
        if self.finPartie(jtrait) or pf == pfmax :
            if self.perdant(jtrait) : 
                return None,True
           
        elif pf != pfmax :
            _adv = self.adversaire(jtrait)
            lfils = self.listeCoups(jtrait)
            for f in lfils :
                if f != self.positionGagnante(pf+1, _adv, pfmax, verbose)[0]:
                    return best[0],False
            return best[0],True
        

        return best[0], False

In [17]:
ordi_IA = IAPlayer(0,1,False)
humain_1 = Humain()
humain_2 = Humain()
jeu = HexVirus(5,5)
partie(jeu, humain_1, ordi_IA)

REGLES : 
1) Chaque joueur, à tour de role, peut poser un pion sur une des cases vides adjacentes à un de ses pions.
2) Lorsqu'un pion d'une couleur est posé à un endroit, tous les pions voisins de la couleur ennemi sont capturés 
(transformés dans la couleur du pion posé).
3) La partie s'arrete lorsque toutes les cases vides sont remplies ou lorsque les joueurs n'ont plus de coups jouables. 

Nouvelle manche : 

joueur 1: ☼ joueur 2:  ☻

       ☼   1   2   3   ☻
     5   6   7   8   9   10
   11   12   13   14   15   16   17
     18   19   20   21   22   23
       ☻   25   26   27   ☼

Le joueur ☼ a le trait
Choisissez une case dans la liste suivante : 
 1  5  6  22  23  27 


KeyboardInterrupt: ignored